# Notebook Goal & Approach

## Goal
For each FERC 714 respondent that reports hourly demand as an electricity planning area, create a geometry representing the geographic area in which that electricity demand originated. Create a separate geometry for each year in which data is available.

## Approach
* Use the `eia_code` found in the `respondent_id_ferc714` table to link FERC 714 respondents to their corresponding EIA utilities or balancing areas.
* Use the `balancing_authority_eia861` and `sales_eia861` tables to figure out which respondents correspond to what utility or utilities (if a BA), and which states of operation.
* Use the `service_territory_eia861` table to link those combinations of years, utilities, and states of operation to collections of counties.
* Given the FIPS codes of the counties associated with each utility or balancing area in a given year, use geospatial data from the US Census to compile an annual demand area geometry.
* Merge those geometries back in with the `respondent_id_ferc714` table, along with additional EIA balancing area and utility IDs / Codes on a per-year basis.

# Imports & Config

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Standard Libraries:
import dateutil
import logging
import pathlib
import pickle
import re
import sys
import zipfile

# 3rd Party Libraries:
import contextily as ctx
import geopandas
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sqlalchemy as sa

# Local Packages:
import pudl

## Configure Output Formatting

In [ ]:
sns.set()
%matplotlib inline

In [ ]:
mpl.rcParams['figure.figsize'] = (20,8)
mpl.rcParams['figure.dpi'] = 150
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

## Logging

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
log_format = '%(asctime)s [%(levelname)8s] %(name)s:%(lineno)s %(message)s'
formatter = logging.Formatter(log_format)
handler.setFormatter(formatter)
logger.handlers = [handler]

## PUDL Setup

In [ ]:
pudl_settings = pudl.workspace.setup.get_defaults()
ferc1_engine = sa.create_engine(pudl_settings['ferc1_db'])
pudl_engine = sa.create_engine(pudl_settings['pudl_db'])
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine)
pudl_settings

# Parameters

In [ ]:
MAP_CRS = "EPSG:3857"
CALC_CRS = "ESRI:102003"

# Function Definitions

## Dummy EIA 861 ETL

In [ ]:
def test_etl_eia(eia_inputs, pudl_settings):
    """
    This is a dummy function that runs the first part of the EIA ETL
    process -- everything up until the entity harvesting begins. For
    use in this notebook only.

    """
    eia860_tables = eia_inputs["eia860_tables"]
    eia860_years = eia_inputs["eia860_years"]
    eia861_tables = eia_inputs["eia861_tables"]
    eia861_years = eia_inputs["eia861_years"]
    eia923_tables = eia_inputs["eia923_tables"]
    eia923_years = eia_inputs["eia923_years"]

    # generate CSVs for the static EIA tables, return the list of tables
    #static_tables = _load_static_tables_eia(datapkg_dir)

    # Extract EIA forms 923, 860
    eia860_raw_dfs = pudl.extract.eia860.Extractor().extract(eia860_years, testing=True)
    eia861_raw_dfs = pudl.extract.eia861.Extractor().extract(eia861_years, testing=True)
    eia923_raw_dfs = pudl.extract.eia923.Extractor().extract(eia923_years, testing=True)

    # Transform EIA forms 860, 861, 923
    eia860_transformed_dfs = pudl.transform.eia860.transform(eia860_raw_dfs, eia860_tables=eia860_tables)
    eia861_transformed_dfs = pudl.transform.eia861.transform(eia861_raw_dfs, eia861_tables=eia861_tables)
    eia923_transformed_dfs = pudl.transform.eia923.transform(eia923_raw_dfs, eia923_tables=eia923_tables)

    # create an eia transformed dfs dictionary
    eia_transformed_dfs = eia860_transformed_dfs.copy()
    eia_transformed_dfs.update(eia861_transformed_dfs.copy())
    eia_transformed_dfs.update(eia923_transformed_dfs.copy())

    # convert types..
    eia_transformed_dfs = pudl.helpers.convert_dfs_dict_dtypes(eia_transformed_dfs, 'eia')

    return eia_transformed_dfs

## Dummy EIA 861 Harvesting
* Used to separately test the EIA entity harvesting process with EIA 861
* Doesn't yet work b/c 861 is structured differently than 860/923.

In [ ]:
def test_harvest_eia(eia_transformed_dfs, eia860_years, eia861_years, eia923_years):
    entities_dfs, eia_transformed_dfs = pudl.transform.eia.transform(
        eia_transformed_dfs,
        eia860_years=eia860_years,
        eia861_years=eia861_years,
        eia923_years=eia923_years,
    )

    # convert types..
    entities_dfs = pudl.helpers.convert_dfs_dict_dtypes(entities_dfs, 'eia')

    # Compile transformed dfs for loading...
    return entities_dfs, eia_transformed_dfs

## Compare Annual Demand vs. Sales

In [ ]:
def annual_demand_vs_sales(dhpa_ferc714, sales_eia861, ba_eia861):
    """
    Categorize EIA Codes in FERC 714 as BA or Utility IDs.
    
    Most FERC 714 respondent IDs are associated with an `eia_code` which
    refers to either a `balancing_authority_id_eia` or a `utility_id_eia`
    but no indication is given as to which type of ID each one is. This
    is further complicated by the fact that EIA uses the same numerical
    ID to refer to the same entity in most but not all cases, when that
    entity acts as both a utility and as a balancing authority.
    
    In order to identify which type of ID each `eia_code` is, this
    funciton compares the annual demand reported in association with
    each code in the FERC 714 hourly planning area time series, and in
    the EIA 861 sales table -- using the ID both as a utility and as a
    balancing authority ID. The correlation between the FERC 714 demand
    and the EIA 861 sales should be much higher for one type of ID than
    the other, indicating which type of ID is represented in the FERC
    714 data.
    
    Args:
        dhpa_ferc714 (pandas.DataFrame): The FERC 714 hourly demand
            time series.
        sales_eia861 (pandas.DataFrame): The EIA 861 Sales table.
        ba_eia861 (pandas.DataFrame): The EIA 861 Balancing Authority
            table, which contains the mapping between EIA Balancing
            Authority Codes (3-4 letters) and EIA Balancing Authority
            IDs (integers). The codes are present in the Sales table,
            but the IDs are what the eia_code refers to.
    
    Returns:
        pandas.DataFrame: A table containing FERC 714 respondent IDs,
        EIA codes, and a column indicating whether that code was
        found to be more consistent with Balancing Authority or
        Utility electricity demand / sales.
    
    """
    # Sum up FERC 714 demand by report_year and eia_code:
    dhpa_ferc714_by_eia_code = (
        dhpa_ferc714
        .groupby(["eia_code", "report_year"])["demand_mwh"]
        .sum()
        .reset_index()
    ) 

    # Sum up the EIA 861 sales by Utility ID:
    sales_eia861_by_util = (
        sales_eia861.groupby(["utility_id_eia", "report_date"])["sales_mwh"]
        .sum()
        .reset_index()
        .assign(report_year=lambda x: x.report_date.dt.year)
        .drop("report_date", axis="columns")
        .rename(columns={"sales_mwh": "sales_utility_mwh"})
    )

    # Need to translate the BA Code to BA ID for comparison w/ eia_code
    ba_codes_and_ids = (
        ba_eia861[["balancing_authority_code_eia", "balancing_authority_id_eia", "report_date"]]
        .drop_duplicates()
        .assign(report_year=lambda x: x.report_date.dt.year)
        .drop("report_date", axis="columns")
        .dropna()
    )

    # Sum up the EIA 861 sales by Balancing Authority Code:
    sales_eia861_by_ba = (
        sales_eia861
        .groupby(["balancing_authority_code_eia", "report_date"], observed=True)["sales_mwh"]
        .sum()
        .reset_index()
        .assign(report_year=lambda x: x.report_date.dt.year)
        .drop("report_date", axis="columns")
        .rename(columns={"sales_mwh": "sales_ba_mwh"})
        .query("balancing_authority_code_eia!='UNK'")
        .merge(ba_codes_and_ids)
    )
    # Combine the demand and sales data with all the IDs
    demand_and_sales = (
        dhpa_ferc714_by_eia_code
        .merge(
            sales_eia861_by_util,
            left_on=["eia_code", "report_year"],
            right_on=["utility_id_eia", "report_year"],
            how="left"
        )
        .merge(
            sales_eia861_by_ba,
            left_on=["eia_code", "report_year"],
            right_on=["balancing_authority_id_eia", "report_year"],
            how="left"
        )
        .astype({
            "eia_code": pd.Int64Dtype(),
            "utility_id_eia": pd.Int64Dtype(),
            "balancing_authority_id_eia": pd.Int64Dtype(),
        })
        .assign(
            ba_ratio=lambda x: x.sales_ba_mwh / x.demand_mwh,
            utility_ratio=lambda x: x.sales_utility_mwh / x.demand_mwh,
        )
    )
    return demand_and_sales

## EIA Code Categorization

In [ ]:
def categorize_eia_code(rids_ferc714, utils_eia860, ba_eia861):
    """
    Categorize EIA Codes in FERC 714 as BA or Utility IDs.
    
    Most FERC 714 respondent IDs are associated with an `eia_code` which
    refers to either a `balancing_authority_id_eia` or a `utility_id_eia`
    but no indication is given as to which type of ID each one is. This
    is further complicated by the fact that EIA uses the same numerical
    ID to refer to the same entity in most but not all cases, when that
    entity acts as both a utility and as a balancing authority.
    
    Given the nature of the FERC 714 hourly demand dataset, this function
    assumes that if the `eia_code` appears in the EIA 861 Balancing
    Authority table, that it should be labeled `balancing_authority`.
    If the `eia_code` appears only in the EIA 860 Utility table, then
    it is labeled `utility`. These labels are put in a new column named
    `respondent_type`. If the planning area's `eia_code` does not appear in
    either of those tables, then `respondent_type is set to NA.

    Args:
        rids_ferc714 (pandas.DataFrame): The FERC 714 `respondent_id` table.
        utils_eia860 (pandas.DataFrame): The EIA 860 Utilities output table.
        ba_eia861 (pandas.DataFrame): The EIA 861 Balancing Authority table.
    
    Returns:
        pandas.DataFrame: A table containing all of the columns present in
        the FERC 714 `respondent_id` table, plus  a new one named
        `respondent_type` which can take on the values `balancing_authority`,
        `utility`, or the special value pandas.NA.

    """
    ba_ids = set(ba_eia861.balancing_authority_id_eia.dropna())
    util_not_ba_ids = set(utils_eia860.utility_id_eia.dropna()).difference(ba_ids)
    new_rids = rids_ferc714.copy()
    new_rids["respondent_type"] = pd.NA
    new_rids.loc[new_rids.eia_code.isin(ba_ids), "respondent_type"] = "balancing_authority"
    new_rids.loc[new_rids.eia_code.isin(util_not_ba_ids), "respondent_type"] = "utility"
    ba_rids = new_rids[new_rids.respondent_type=="balancing_authority"]
    util_rids = new_rids[new_rids.respondent_type=="utility"]
    na_rids = new_rids[new_rids.respondent_type.isnull()]

    ba_rids = (
        ba_rids.merge(
            ba_eia861
            .filter(like="balancing_")
            .drop_duplicates(subset=["balancing_authority_id_eia", "balancing_authority_code_eia"]),
            how="left", left_on="eia_code", right_on="balancing_authority_id_eia"
        )
    )
    util_rids = (
        util_rids.merge(
            utils_eia860[["utility_id_eia", "utility_name_eia"]]
            .drop_duplicates("utility_id_eia"),
            how="left", left_on="eia_code", right_on="utility_id_eia"
        )
    )
    new_rids = (
        pd.concat([ba_rids, util_rids, na_rids])
        .astype({
            "respondent_type": pd.StringDtype(),
            "balancing_authority_code_eia": pd.StringDtype(),
            "balancing_authority_id_eia": pd.Int64Dtype(),
            "balancing_authority_name_eia": pd.StringDtype(),
            "utility_id_eia": pd.Int64Dtype(),
            "utility_name_eia": pd.StringDtype(),
        })
    )
    
    return new_rids

## Georeference Balancing Authorities

In [ ]:
def georef_bas(ba_eia861, st_eia861, sales_eia861, census_gdf):
    """
    Create a GeoDataFrame mapping BAs to Utils to county geometries by year.
    
    This GDF includes the following columns:
    
    balancing_authority_id_eia (ba_eia861)
    balancing_authority_name_eia (ba_eia861)
    balancing_authority_code_eia (ba_eia861)
    utility_id_eia (sales_eia861)
    utility_name_eia (sales_eia861)
    county_id_fips (st_eia861)
    county (st_eia861)
    state_id_fips (st_eia861)
    state (st_eia861)
    geometry (census_gdf)
    county_name_census (census_gdf)
    
    It includes information both about which counties are associated with
    utilities that are part of balancing authorities, and utilities that
    are not part part of balancing authorities, so should be possible to
    use it to generate geometries for all of the respondents in FERC 714,
    both BAs and Utils.

    """
    # Make sure that there aren't any more BA IDs we can recover from later years:
    ba_ids_missing_codes = (
        ba_eia861.loc[ba_eia861.balancing_authority_code_eia.isnull(), "balancing_authority_id_eia"]
        .drop_duplicates()
        .dropna()
    )
    assert len(ba_eia861[
        (ba_eia861.balancing_authority_id_eia.isin(ba_ids_missing_codes)) &
        (ba_eia861.balancing_authority_code_eia.notnull())
    ]) == 0
    
    # Which utilities were part of what balancing areas in 2010-2012?
    early_ba_by_util = (
        ba_eia861
        .query("report_date <= '2012-12-31'")
        .loc[:, [
            "report_date",
            "balancing_authority_id_eia",
            "balancing_authority_code_eia",
            "utility_id_eia",
            "balancing_authority_name_eia",
        ]]
        .drop_duplicates(subset=["report_date", "balancing_authority_id_eia", "utility_id_eia"])
    )

    # Create a dataframe that associates utilities and balancing authorities.
    # This information is directly avaialble in the early_ba_by_util dataframe
    # but has to be compiled for 2013 and later years based on the utility
    # BA associations that show up in the Sales table
    # Create an annual, normalized version of the BA table:
    ba_normed = (
        ba_eia861
        .loc[:, [
            "report_date",
            "state",
            "balancing_authority_code_eia",
            "balancing_authority_id_eia",
            "balancing_authority_name_eia",
        ]]
        .drop_duplicates(subset=[
            "report_date",
            "state",
            "balancing_authority_code_eia",
            "balancing_authority_id_eia",
        ])
    )
    ba_by_util = (
        pd.merge(
            ba_normed,
            sales_eia861
            .loc[:, [
                "report_date",
                "state",
                "utility_id_eia",
                "balancing_authority_code_eia"
            ]].drop_duplicates()
        )
        .loc[:, [
            "report_date",
            "state",
            "utility_id_eia",
            "balancing_authority_id_eia"
        ]]
        .append(early_ba_by_util[["report_date", "utility_id_eia", "balancing_authority_id_eia"]])
        .drop_duplicates()
        .merge(ba_normed)
        .dropna(subset=["report_date", "utility_id_eia", "balancing_authority_id_eia"])
        .sort_values(["report_date", "balancing_authority_id_eia", "utility_id_eia", "state"])
    )
    # Merge in county FIPS IDs for each county served by the utility from
    # the service territory dataframe. We do an outer merge here so that we
    # retain any utilities that are not part of a balancing authority. This
    # lets us generate both BA and Util maps from the same GeoDataFrame
    # We have to do this separately for the data up to 2012 (which doesn't
    # include state) and the 2013 and onward data (which we need to have
    # state for)
    early_ba_util_county = (
        ba_by_util.drop("state", axis="columns")
        .merge(st_eia861, on=["report_date", "utility_id_eia"], how="outer")
        .query("report_date <= '2012-12-31'")
    )
    late_ba_util_county = (
        ba_by_util
        .merge(st_eia861, on=["report_date", "utility_id_eia", "state"], how="outer")
        .query("report_date >= '2013-01-01'")
    )
    ba_util_county = pd.concat([early_ba_util_county, late_ba_util_county])
    # Bring in county geometry information based on FIPS ID from Census
    ba_util_county_gdf = (
        census_gdf[["GEOID10", "NAMELSAD10", "geometry"]]
        .to_crs(MAP_CRS)
        .rename(
            columns={
                "GEOID10": "county_id_fips",
                "NAMELSAD10": "county_name_census",
                }
        )
        .merge(ba_util_county)
    )
    
    return ba_util_county_gdf

## Map Balancing Authorities

In [ ]:
def map_ba(ba_ids, year, ba_util_county_gdf, save=False):
    """
    Create a map of a balancing authority for a historical year.
    
    Args:
        ba_ids (iterable): A collection of Balancing Authority IDs.
        year (int): The year for which to create a map.
        ba_util_county_gdf (geopandas.GeoDataFrame): A dataframe
            associating report_date, balancing_authority_id_eia, and
            county_id_fips.
        save (bool): If True, save the figure to disk.
    
    Returns:
        None
    
    """
    map_gdf = (
        ba_util_county_gdf[
            (ba_util_county_gdf.report_date.dt.year == year) &
            (ba_util_county_gdf.balancing_authority_id_eia.isin(ba_ids)) &
            (~ba_util_county_gdf.county_id_fips.str.match("^02")) & # Avoid Alaska
            (~ba_util_county_gdf.county_id_fips.str.match("^15")) & # Avoid Hawaii
            (~ba_util_county_gdf.county_id_fips.str.match("^72")) # Avoid Puerto Rico
        ]
        .drop_duplicates(subset=["balancing_authority_id_eia", "county_id_fips"])
    )
    ax = map_gdf.plot(figsize=(20, 20), color="black", alpha=0.25, linewidth=0.25)
    plt.title(f"Balancing Areas ({year=})")
    ctx.add_basemap(ax)
    if save is True:
        plt.savefig(f"BA_Overlap_{year}.jpg")

In [ ]:
def compare_hifld_eia_ba(ba_code, hifld_gdf, eia_gdf):
    """
    Compare historical EIA BAs vs. HIFLD geometries.
    """
    fig, (hifld_ax, eia_ax) = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True)
    hifld_ax.set_title(f"{ba_code} (HIFLD)")
    hifld_gdf[hifld_gdf.ABBRV==ba_code].to_crs(MAP_CRS).plot(ax=hifld_ax, linewidth=0)

    eia_ax.set_title(f"{ba_code} (EIA)")
    eia_gdf[
        (eia_gdf.balancing_authority_code_eia==ba_code) &
        (eia_gdf.report_date.dt.year == 2017)
    ].plot(ax=eia_ax, linewidth=0.1)
    plt.show()

# Read Data

## EIA 860 via PUDL Outputs

In [ ]:
plants_eia860 = pudl_out.plants_eia860()
utils_eia860 = pudl_out.utils_eia860()

## EIA 861 (2010-2018)
* Not yet fully integrated into PUDL
* Post-transform harvesting process isn't compatible w/ EIA 861 structure
* Only getting the `sales_eia861`, `balancing_authority_eia861`, and `service_territory_eia861` tables

In [ ]:
%%time
logger.setLevel("WARN")
eia_years = list(range(2010, 2019))
eia_inputs = {
    "eia860_years": [],
    "eia860_tables": pudl.constants.pudl_tables["eia860"],
    "eia861_years": eia_years,
    "eia861_tables": pudl.constants.pudl_tables["eia861"],
    "eia923_years": [],
    "eia923_tables": pudl.constants.pudl_tables["eia923"],
}
eia_transformed_dfs = test_etl_eia(eia_inputs=eia_inputs, pudl_settings=pudl_settings)
logger.setLevel("INFO")

In [ ]:
ba_eia861 = eia_transformed_dfs["balancing_authority_eia861"].copy()
st_eia861 = eia_transformed_dfs["service_territory_eia861"].copy()
sales_eia861 = eia_transformed_dfs["sales_eia861"].copy()

In [ ]:
raw_eia861_dfs = pudl.extract.eia861.Extractor().extract(years=range(2010,2019), testing=True)

## FERC 714 (2006-2018)

In [ ]:
%%time
logger.setLevel("WARN")
raw_ferc714 = pudl.extract.ferc714.extract(pudl_settings=pudl_settings)
tfr_ferc714 = pudl.transform.ferc714.transform(raw_ferc714)
logger.setLevel("INFO")

## HIFLD Electricity Planning Areas (2018)
* Electricty Planning Area geometries from HIFLD.
* Indexed by `ID` which corresponds to EIA utility or balancing area IDs.
* Only valid for 2017-2018.

In [ ]:
hifld_pa_gdf = (
    pudl.analysis.demand_mapping.get_hifld_planning_areas_gdf(pudl_settings)
    .to_crs(MAP_CRS)
)

## US Census DP1 (2010)
* This GeoDataFrame contains county-level geometries and demographic data.

In [ ]:
%%time
census_gdf = (
    pudl.analysis.demand_mapping.get_census2010_gdf(pudl_settings, layer="county")
    .to_crs(MAP_CRS)
)

# Combine Data

## Categorize FERC 714 Respondent IDs

In [ ]:
rids_ferc714 = (
    tfr_ferc714["respondent_id_ferc714"]
    .pipe(categorize_eia_code, utils_eia860, ba_eia861)
)

## Add FERC 714 IDs to HIFLD

In [ ]:
hifld_pa_gdf = (
    hifld_pa_gdf
    .merge(rids_ferc714, left_on="ID", right_on="eia_code", how="left")
)

## Add Respondent info to FERC 714 Demand

In [ ]:
dhpa_ferc714 = pd.merge(
    tfr_ferc714["demand_hourly_pa_ferc714"],
    tfr_ferc714["respondent_id_ferc714"],
    on="respondent_id_ferc714",
    how="left",  # There are respondents with no demand
)

# Utilities vs. Balancing Authorities
Exploration of the Balancing Authority EIA 861 table for cleanup

### Which columns are available in which years?

| Year | BA ID | BA Name | BA Code | Util ID | Util Name | State | N  |
|------|-------|---------|---------|---------|-----------|-------|----|
| 2010 | XXXXX | XXXXXXX |         | XXXXXXX |           |       |3193|
| 2011 | XXXXX | XXXXXXX |         | XXXXXXX |           |       |3126|
| 2012 | XXXXX | XXXXXXX |         | XXXXXXX | XXXXXXXXX |       |3146|
| 2013 | XXXXX | XXXXXXX | XXXXXXX |         |           | XXXXX | 239|
| 2014 | XXXXX | XXXXXXX | XXXXXXX |         |           | XXXXX | 208|
| 2015 | XXXXX | XXXXXXX | XXXXXXX |         |           | XXXXX | 203|
| 2016 | XXXXX | XXXXXXX | XXXXXXX |         |           | XXXXX | 203|
| 2017 | XXXXX | XXXXXXX | XXXXXXX |         |           | XXXXX | 203|
| 2018 | XXXXX | XXXXXXX | XXXXXXX |         |           | XXXXX | 204|

### What does this table mean?
* In 2010-2012, the table says which utilities (by ID) are included in which balancing authorities.
* In 2013-2018, the table indicates which *states* a BA is operating in, and also provides a BA Code

### Questions:
* Where does the `balancing_authority_code` show up elsewhere in the EIA 860/861 data?
  * `plants_eia860` (nowhere else that I know of)
* Are the BA to Utility mappings likely to remain valid throughout the entire time period? Can we propagate them forward?
  * No, there's some variation year to year in which utilities are associated with which BAs
* Are the BA Code/Name to BA ID mappings permanent?
  * No they aren't -- when a BA changes owners and names, the code changes, but ID stays the same.

## Untangling HIFLD, FERC 714, & EIA IDs
* There are unspecified "EIA codes" associated with FERC 714 respondents.
* These IDs correspond to a mix of `utility_id_eia` and `balancing_authority_id_eia` values.
* Similarly, the ID field of the HIFLD geometries are a mix of BA and Utility IDs from EIA.
* This is extra confusing, because EIA *usually* uses the *same* ID for BAs and Utils.
* However, the EIA BA and Util IDs appear to be distinct namespaces
  * Not all IDs which appear in both tables identify the same entity in both tables.
  * In a few cases different IDs are used to identify the same entity when it shows up in both tables.
* It could be that whoever entered the IDs in the FERC 714 / HIFLD datasets didn't realize these were different sets of IDs.

### BA / Utility ID Overlap
* Example of an ID that shows up in both, but refers to different entities, see `59504`
  * `balancing_area_id_eia == 59504` is the Southwest Power Pool (SWPP).
  * `utility_id_eia == 59504` is Kirkwood Community College, in MO.
* Example of an entity that exists in both datsets, but shows up with different IDs, see PacifiCorp.
  * Has two BA IDs (East and West): `[14379, 14378]`
  * Has one Utility ID: `14354`
* Example of an entity that shows up with the same ID in both tables:
  * ID `15466` is Public Service Co of Colorado -- both a BA (PSCO) and a Utility.

In [ ]:
# BA ID comes from EIA 861 BA Table
ba_ids = set(ba_eia861.balancing_authority_id_eia)
print(f"Total # of BA IDs: {len(ba_ids)}")

# Util ID comes from EIA 860 Utilities Entity table.
util_ids = set(pudl_out.utils_eia860().utility_id_eia)
print(f"Total # of Util IDs: {len(util_ids)}")

ba_not_util_ids = ba_ids.difference(util_ids)
print(f"BA IDs that are not Util IDs: {len(ba_not_util_ids)}")

util_not_ba_ids = util_ids.difference(ba_ids)
print(f"Util IDs that are not BA IDs: {len(util_not_ba_ids)}")

ba_and_util_ids = ba_ids.intersection(util_ids)
print(f"BA IDs that are also Util IDs: {len(ba_and_util_ids)}")

In [ ]:
ba_and_util = (
    ba_eia861
    .loc[:, ["balancing_authority_id_eia", "balancing_authority_name_eia"]]
    .dropna(subset=["balancing_authority_id_eia"])
    .merge(
        pudl_out.utils_eia860(),
        left_on="balancing_authority_id_eia",
        right_on="utility_id_eia",
        how="inner"
    )
    .loc[:, [
        "utility_id_eia",
        "balancing_authority_name_eia",
        "utility_name_eia",
    ]]
    .rename(columns={"utility_id_eia": "util_ba_id"})
    .drop_duplicates()
    .reset_index(drop=True)
)

ba_not_util = (
    ba_eia861.loc[ba_eia861.balancing_authority_id_eia.isin(ba_not_util_ids)]
    .loc[:,["balancing_authority_id_eia", "balancing_authority_code_eia", "balancing_authority_name_eia"]]
    .drop_duplicates(subset=["balancing_authority_id_eia", "balancing_authority_code_eia"])
    .sort_values("balancing_authority_id_eia")
)

### Missing IDs
* There are `eia_code` values that don't show up in the list of balancing authority IDs (2010-2018).
* There are also `eia_code` values that don't show up in the list of utility IDs (2009-2018).
* There are a few `eia_code` values that don't show up in either!
* Mostly this is an artifact of the different time covered by FERC 714 (2006-2018).
* If we look only at the respondents that reported non-zero demand for 2010-2018, we find that all of the `eia_code` values *do* appear in either the `blancing_authority_eia861` or `utilities_eia860` tables.

In [ ]:
rids_ferc714[
    (~rids_ferc714.eia_code.isin(ba_eia861.balancing_authority_id_eia.unique())) &
    (~rids_ferc714.eia_code.isin(utils_eia860.utility_id_eia.unique()))
]

In [ ]:
rids_recent = (
    dhpa_ferc714
    .groupby(["respondent_id_ferc714", "report_year"])
    .agg({"demand_mwh": sum})
    .reset_index()
    .query("report_year >= 2010")
    .query("demand_mwh >= 0.0")
    .merge(rids_ferc714[["eia_code", "respondent_id_ferc714", "respondent_name_ferc714"]], how="left")
    .drop(["report_year", "demand_mwh"], axis="columns")
    .drop_duplicates()
)
assert len(rids_recent[
    (~rids_recent.eia_code.isin(ba_eia861.balancing_authority_id_eia.unique())) &
    (~rids_recent.eia_code.isin(utils_eia860.utility_id_eia.unique()))
]) == 0

### BA to Utility Mappings are Many to Many
* Unsurprisingly, BAs often contain many utilities.
* However, it's also common for utilities to participate in more than one BA.
* About 1/3 of all utilities show up in association with more than one BA

In [ ]:
ba_to_util_mapping = (
    ba_eia861[["balancing_authority_id_eia", "utility_id_eia"]]
    .dropna(subset=["balancing_authority_id_eia", "utility_id_eia"])
    .drop_duplicates(subset=["balancing_authority_id_eia", "utility_id_eia"])
    .groupby(["balancing_authority_id_eia"])
    .agg({
        "utility_id_eia": "count"
    })
)
plt.hist(ba_to_util_mapping.utility_id_eia, bins=99, range=(1,100))
plt.xlabel("# of Utils / BA")
plt.ylabel("# of BAs")
plt.title("Number of Utilities per Balancing Area");

In [ ]:
util_to_ba_mapping = (
    ba_eia861[["balancing_authority_id_eia", "utility_id_eia"]]
    .dropna(subset=["balancing_authority_id_eia", "utility_id_eia"])
    .drop_duplicates(subset=["balancing_authority_id_eia", "utility_id_eia"])
    .groupby(["utility_id_eia"])
    .agg({
        "balancing_authority_id_eia": "count"
    })
)
plt.hist(util_to_ba_mapping.balancing_authority_id_eia, bins=4, range=(1,5))
plt.title("Number of Balancing Authorities per Utility");

## Georeferenced Demand Fraction
* With their original EIA codes the HIFLD Electricity Planning Areas only georeference some of the FERC 714 demand.
* It's about 86% in 2018. In 2013 and earlier years, the fraction starts to drop off more quickly, to 76% in 2010, and 58% in 2006.
* After manually identifying and fixing some bad and missing EIA codes in the FERC 714, the mapped fraction is much higher.
* 98% or more in 2014-2018, dropping to 87% in 2010, and 68% in 2006
* **However** because the geometries have also evolved over time, just the fact that the demand time series is linked to **some** HIFLD geometry, doesn't mean that it's the **right** geometry.

In [ ]:
annual_demand_ferc714 = (
    dhpa_ferc714
    .groupby(["report_year"]).demand_mwh.sum()
    .reset_index()
)
annual_demand_mapped = (
    dhpa_ferc714[dhpa_ferc714.eia_code.isin(hifld_pa_gdf.eia_code)]
    .groupby(["report_year"]).demand_mwh.sum()
    .reset_index()
    .merge(annual_demand_ferc714, on="report_year", suffixes=("_map", "_tot"))
    .assign(
        fraction_mapped=lambda x: x.demand_mwh_map / x.demand_mwh_tot
    )
)

In [ ]:
plt.plot("report_year", "fraction_mapped", data=annual_demand_mapped, lw=5)
plt.ylabel("Fraction of demand which is mapped")
plt.title("Completeness of HIFLD demand mapping by year")
plt.ylim(0.6, 1.05);

# Historical Planning Area Geometries
Compile a GeoDataFrame that relates balancing authorities, their constituent utilities, and the collections of counties which are served by those utilities, across all the years for which we have EIA 861 data (2010-2018)

In [ ]:
ba_util_county_gdf = georef_bas(ba_eia861, st_eia861, sales_eia861, census_gdf)

In [ ]:
ba_util_county_gdf.info()

In [ ]:
for year in (2010, 2014, 2018):
    map_ba(ba_util_county_gdf.balancing_authority_id_eia.unique(), year, ba_util_county_gdf, save=True)

## Output Simplified Annual BA Geometries
* This takes half an hour so it's commented out.
* Resulting shapefile is ~250MB compressed. Seems too big.
* Need to figure out how to add explicity projection.
* Need to figure out how to make each year's BA geometries its own layer.

In [ ]:
#%%time
#ba_fips_simplified = (
#    ba_util_county_gdf
#    .assign(report_year=lambda x: x.report_date.dt.year)
#    .drop([
#        "report_date",
#        "state",
#        "state_id_fips",
#        "county",
#        "county_name_census",
#        "utility_id_eia",
#        "utility_name_eia"
#    ], axis="columns")
#    .drop_duplicates(subset=["report_year", "balancing_authority_id_eia", "county_id_fips"])
#    .dropna(subset=["report_year", "balancing_authority_id_eia", "county_id_fips"])
#    .loc[:,["report_year", "balancing_authority_id_eia", "balancing_authority_code_eia", "balancing_authority_name_eia", "county_id_fips", "geometry"]]
#)
#ba_annual_gdf = (
#    ba_fips_simplified
#    .dissolve(by=["report_year", "balancing_authority_id_eia"])
#    .reset_index()
#    .drop("county_id_fips", axis="columns")
#)
#ba_output_gdf = (
#    ba_annual_gdf
#    .astype({
#        "report_year": int,
#        "balancing_authority_id_eia": float,
#        "balancing_authority_code_eia": str,
#        "balancing_authority_name_eia": str,
#    })
#    .rename(columns={
#        "report_year": "year",
#        "balancing_authority_id_eia": "ba_id",
#        "balancing_authority_code_eia": "ba_code",
#        "balancing_authority_name_eia": "ba_name",
#    })
#)
#ba_output_gdf.to_file("ba_annual.shp")

## Compare HIFLD and EIA BA maps for 2018

In [ ]:
for ba_code in hifld_pa_gdf.ABBRV.unique():
    if ba_code in ba_util_county_gdf.balancing_authority_code_eia.unique():
        compare_hifld_eia_ba(ba_code, hifld_pa_gdf, ba_util_county_gdf)

## Time Evolution of BA Geometries
For each BA we now have a collection of annual geometries. How have they changed over time?

In [ ]:
for ba_code in ba_util_county_gdf.balancing_authority_code_eia.unique():
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(20,20), sharex=True, sharey=True, facecolor="white")
    for year, ax in zip(range(2010, 2019), axes.flat):
        ax.set_title(f"{ba_code} ({year})")
        ax.set_xticks([])
        ax.set_yticks([])
        plot_gdf = (
            ba_util_county_gdf
            .assign(report_year=lambda x: x.report_date.dt.year)
            .query(f"balancing_authority_code_eia=='{ba_code}'")
            .query(f"report_year=='{year}'")
            .drop_duplicates(subset="county_id_fips")
        )
        plot_gdf.plot(ax=ax, linewidth=0.1)
    plt.show()

## Merge Geometries with FERC 714
Now that we have a draft of wht the BA and Utility level territories look like, we can merge those with the FERC 714 Respondent ID table, and see how many leftovers there are, and whether the BA and Utility geometires play well together.

Before dissolving the boundaries between counties the output dataframe needs to have:
* `report_date`
* `respondent_id_ferc714`
* `eia_code`
* `respondent_type`
* `balancing_authority_id_eia`
* `utility_id_eia`
* `county_id_fips`
* `geometry`

* `balancing_authority_code_eia`
* `balancing_authority_name_eia`
* `respondent_name_ferc714`
* `utility_name_eia`
* `county_name_census`
* `state`
* `state_id_fips`

In [ ]:
utils_ferc714 = (
    rids_ferc714.loc[
        rids_ferc714.respondent_type == "utility",
        ["respondent_id_ferc714", "respondent_name_ferc714", "utility_id_eia", "respondent_type"]
    ]
)

bas_ferc714 = (
    rids_ferc714.loc[
        rids_ferc714.respondent_type == "balancing_authority",
        ["respondent_id_ferc714", "respondent_name_ferc714", "balancing_authority_id_eia", "respondent_type"]
    ]
)

null_ferc714 = (
    rids_ferc714.loc[
        rids_ferc714.respondent_type.isnull(),
        ["respondent_id_ferc714", "respondent_name_ferc714", "respondent_type"]
    ]
)

bas_ferc714_gdf = (
    ba_util_county_gdf
    .drop(["county"], axis="columns")
    .merge(bas_ferc714, how="right")
)

utils_ferc714_gdf = (
    ba_util_county_gdf
    .drop(["balancing_authority_id_eia", "balancing_authority_code_eia", "balancing_authority_name_eia", "county"], axis="columns")
    .drop_duplicates()
    .merge(utils_ferc714, how="right")
)
rids_ferc714_gdf = (
    pd.concat([bas_ferc714_gdf, utils_ferc714_gdf, null_ferc714])
    .astype({
        "county_id_fips": pd.StringDtype(),
        "county_name_census": pd.StringDtype(),
        "respondent_type": pd.StringDtype(),
        "utility_id_eia": pd.Int64Dtype(),
        "balancing_authority_id_eia": pd.Int64Dtype(),
        "balancing_authority_code_eia": pd.StringDtype(),
        "balancing_authority_name_eia": pd.StringDtype(),
        "state": pd.StringDtype(),
        "utility_name_eia": pd.StringDtype(),
    })
)

In [ ]:
display(rids_ferc714_gdf.info())
rids_ferc714_gdf.sample(10)

## Check Geometries for Completeness
* How many balancing authorities do we have geometries for?
* How many utilities do we have geometries for?
* Do those geometries cover all of the entities that report in FERC 714?
* Do we have a geometry for every entity in every year in which it reports demand?

### Count BA & Util Geometries

In [ ]:
n_bas = len(rids_ferc714_gdf.balancing_authority_id_eia.unique())
logger.info(f"Found territories for {n_bas} unique Balancing Areas")
n_utils = len(rids_ferc714_gdf.loc[
    (rids_ferc714_gdf.balancing_authority_id_eia.isnull()) &
    (~rids_ferc714_gdf.utility_id_eia.isnull())
].utility_id_eia.unique())
logger.info(f"Found territories for {n_utils} Utilities outside of the BAs")

### Identify Missing Geometries
* Within each year of historical data from 2010-2018, are there any entities (either BA or Utility) which **do** have hourly demand reported in the FERC 714, for whivh we do not have a historical geometry?
* How many of them are there?
* Why are they missing?
* Do we have the geometires in adjacent years and can we re-use them?
* Is it possible that the FERC 714 IDs correspond to a precursor entity, or one that was discontinued?  E.g. if SWPP is missing in 2010, is that because the BA was reported in EIA as SPS in that year?
* How important are the missing geometries? Do the associated entities have a lot of demand associated with them in FERC 714?
* Can we use `ffill` or `backfill` on the `geometry` column in a GeoDataFrame?

In [ ]:
problem_ids = pd.DataFrame()
for year in range(2010, 2019):
    this_year_gdf = (
        rids_ferc714_gdf
        .loc[(rids_ferc714_gdf.report_date.dt.year==year) & (~rids_ferc714_gdf.geometry.isnull())]
    )
    # All BA IDs which show up in FERC 714:
    ba_ids_ferc714 = (
        rids_ferc714
        .loc[rids_ferc714.respondent_type=="balancing_authority",
             "balancing_authority_id_eia"]
        .unique()
    )
    # BA IDs which have a geometry in this year
    ba_geom_ids = (
        this_year_gdf
        .balancing_authority_id_eia
        .dropna().unique()
    )
    # BA IDs which have reported demand in this year
    ba_demand_ids = (
        dhpa_ferc714
        .query("report_year==@year")
        .query("demand_mwh>0.0")
        .loc[dhpa_ferc714.eia_code.isin(ba_ids_ferc714)]
        .eia_code.unique()
    )

    # Need to make the demand IDs clearly either utility of BA IDs. Whoops!
    missing_ba_geom_ids = [x for x in ba_demand_ids if x not in ba_geom_ids]
    logger.info(f"{len(missing_ba_geom_ids)} BA respondents w/o geometries in {year}")
    problem_ids = problem_ids.append(
        rids_ferc714
        .loc[rids_ferc714.balancing_authority_id_eia.isin(missing_ba_geom_ids)]
        .assign(year=year)
    )
    
    # All EIA Utility IDs which show up in FERC 714:
    util_ids_ferc714 = (
        rids_ferc714
        .loc[rids_ferc714.respondent_type=="utility",
             "utility_id_eia"]
        .unique()
    )
    # EIA Utility IDs which have geometry information for this year
    util_geom_ids = (
        this_year_gdf
        .utility_id_eia
        .dropna().unique()
    )
    util_demand_ids = (
        dhpa_ferc714
        .query("report_year==@year")
        .query("demand_mwh>0.0")
        .loc[dhpa_ferc714.eia_code.isin(util_ids_ferc714)]
        .eia_code.unique()
    )
    
    missing_util_geom_ids = [x for x in util_demand_ids if x not in util_geom_ids]
    logger.info(f"{len(missing_util_geom_ids)} Utility respondents w/o geometries in {year}")
    problem_ids = problem_ids.append(
        rids_ferc714
        .loc[rids_ferc714.utility_id_eia.isin(missing_util_geom_ids)]
        .assign(year=year)
    )

In [ ]:
problem_ids.query("year==2010").query("respondent_type=='balancing_authority'")

## Dissolve to BA or Util
* At this point we still have geometires at the county level.
* This is 150,000+ records.
* Really we just want a single geometry per respondent per year.
* Dissolve based on year and respondent_id_ferc714.
* Merge the annual per-respondent geometry with the rids_ferc714 which has more information
* Note that this takes about half an hour to run...

In [ ]:
%%time
dissolved_rids_ferc714_gdf = (
    rids_ferc714_gdf.drop_duplicates(subset=["report_date", "county_id_fips", "respondent_id_ferc714"])
    .dissolve(by=["report_date", "respondent_id_ferc714"])
    .reset_index()
    .loc[:, ["report_date", "respondent_id_ferc714", "geometry"]]
    .merge(rids_ferc714, on="respondent_id_ferc714", how="outer")
)
#dissolved_rids_ferc714_gdf.to_file("planning_areas_ferc714.gpkg", driver="GPKG")

### Select based on respondent type

In [ ]:
dissolved_utils = dissolved_rids_ferc714_gdf.query("respondent_type=='utility'")
dissolved_bas = dissolved_rids_ferc714_gdf.query("respondent_type=='balancing_authority'")

### Nationwide BA / Util Maps
* Still want to add the US state boundaries / coastlines to this for context.

In [ ]:
unwanted_ba_ids = (
    112, # Alaska
    133, # Alaska
    178, # Hawaii
    301, # PJM Dupe
    302, # PJM Dupe
    303, # PJM Dupe
    304, # PJM Dupe
    305, # PJM Dupe
    306, # PJM Dupe
)

for report_date in pd.date_range(start="2010-01-01", end="2018-01-01", freq="AS"):
    ba_ax = (
        dissolved_bas
        .query("report_date==@report_date")
        .query("respondent_id_ferc714 not in @unwanted_ba_ids")
        .plot(figsize=(20, 20), color="blue", alpha=0.25, linewidth=1)
    )
    plt.title(f"FERC 714 Balancing Authority Respondents {report_date}")
    ctx.add_basemap(ba_ax)

    util_ax = (
        dissolved_utils
        .query("report_date==@report_date")
        .plot(figsize=(20, 20), color="red", alpha=0.25, linewidth=1)
    )
    plt.title(f"FERC 714 Utility Respondents {report_date}")
    ctx.add_basemap(util_ax)
    
    plt.show();

### Per-respondent annual maps
* For each respondent make a grid of 9 (2010-2018)
* Show state lines in bg for context
* Limit bounding box by the respondent's territory

# Remaining Tasks

## Geometry Cleanup:
* Why do some respondents lack geometries in some years?
* Why do some respondents lack geometries in **all** years? (e.g. Tri-State G&T)
* Why do some counties have no BA or Utility coverage in some or all years?
* What combinations of years and respondents are missing?
* Compare what we've ended up doing to the Aufhammer paper again.
* Is there any need to use name-based matching between the Planning Area descriptions & EIA Service Territories?
* Problem BAs / Utilities:
  * All the WAPA BAs
  * PacifiCorp East / West
  * Southern Company
  * MISO (Some other IDs that seem related?)
  * PJM (Early years seem out of bounds)

## FERC 714 Demand Time Series Cleanup

### Find broken data:
* Run Tyler Ruggles' anomaly detection code as improved by Greg Schivley
* What kind of anomalies are we finding? Are they a problem? What portion of the overall dataset do they represent?

### Repair data:
* How do we want to fill in the gaps?
* Ideally would be able to use the MICE technique that Tyler used, but we need to keep it all in Python.
* Can do much simpler rolling averages or something for the moment when there are small gaps just to have completeness.
* Should make this gap filling process modular -- use different techniques and see whether they do what we need.

# Miscellaneous Notes

## FERC 714 Demand Irregularities
Unusual issues that need to be addressed, or demand discontinuities that may be useful in the context of aggregating historical demand into modern planning areas. Organized by FERC 714 Respondent ID:

* Missing demand data / weird zeroes
  * 111: (2008)
  * 125: (2015)
  * 137: (2006)
  * 139: (2006) Only the last hour of every day. Maybe 0-23 vs 1-24 reporting?
  * 141: (2006, 2007, 2008, 2009, 2010)
  * 148: (2006)
  * 153: (2006)
  * 154: (2006)
  * 161: (all)
  * 183: (2007, 2009)
  * 208: (2008)
  * 273: (2007, 2008)
  * 283: (2007)
  * 287: (2008-2012)
  * 288: (2006)
  * 289: (2009)
  * 293: (2006)
  * 294: (2006)
  * 311: (2008-2011)
* Inverted Demand (Sign Errors):
  * 156: (2006, 2007, 2008, 2009)
  * 289: (2006-2008, 2010)
* Large demand discontinuities
  * 107: Demand triples at end of 2006.
  * 115: Two big step downs, 2007-2008, and 2011-2012
  * 121: 50% increase at end of 2007.
  * 128: Step up at end of 2007
  * 133: Step down end of 2013 and again end of 2015
  * 190: Demand doubled at end of 2008
  * 214: 50% jump in early 2012.
  * 256: big jump at end of 2006.
  * 261: Big jump at end of 2008.
  * 274: drop at end of 2007
  * 275: Jump at end of 2007
  * 287: Demand before and after big gap are very different.
  * 299: Big drop at end of 2015
  * 307: Jump at end of 2014
  * 321: Jump at end of 2013